In [1]:
import numpy as np
from numpy import array
from pickle import dump, load
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import Dense, LSTM, Embedding
from keras.optimizers import Adadelta

from txt2sequence import convert_to_sequences

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Convert Text to Sequences
N.B. If 'texts/hp_sequences.txt' has already been generated you don't need to run the cell below

In [2]:
convert_to_sequences()

importing books
cleaning text
removing infrequent words
saving sequences to file


# Train Language Model

In [34]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'texts/hp_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [36]:
vocab_size = len(tokenizer.word_index) + 1
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [37]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 64)            528832    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 8263)              1065927   
Total params: 1,841,671
Trainable params: 1,841,671
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
# compile model
optimizer = Adadelta()

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# fit model

In [39]:
model.fit(X, y, batch_size=128, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1176639/1176639 [==============================] - 2301s 2ms/step - loss: 6.0407 - acc: 0.0990
Epoch 2/100
1176639/1176639 [==============================] - 2370s 2ms/step - loss: 5.4109 - acc: 0.1503
Epoch 3/100
1176639/1176639 [==============================] - 2341s 2ms/step - loss: 5.2561 - acc: 0.1661
Epoch 4/100
1176639/1176639 [==============================] - 2226s 2ms/step - loss: 5.1843 - acc: 0.1745
Epoch 5/100
1176639/1176639 [==============================] - 2317s 2ms/step - loss: 5.1356 - acc: 0.1811
Epoch 6/100
1176639/1176639 [==============================] - 2332s 2ms/step - loss: 5.0987 - acc: 0.1867
Epoch 7/100
1176639/1176639 [==============================] - 2330s 2ms/step - loss: 5.0686 - acc: 0.1910
Epoch 8/100
1176639/1176639 [==============================] - 2333s 2ms/step - loss: 5.0479 - acc: 0.1944
Epoch 9/100
1176639/1176639 [==============================] - 2335s 2ms/step - loss: 5.0269 - a

In [40]:
model.save('models/251019/model.h5')
with open('models/251019/model.json', 'w') as json_file:
    json_file.write(model.to_json())
# save the tokenizer
dump(tokenizer, open('models/251019/tokenizer.pkl', 'wb'))